# Fit simulation results to experiment

<div id="toc-wrapper"><h3> Table of Contents </h3><div id="toc" style="max-height: 787px;"><ol class="toc-item"><li><a href="#Globals">Globals</a></li><li><a href="#Fit-Run-2-stall-strength-to-reproduce-measured-single-mutant-YFP-rates-for-Run-3-initiation-mutant-simulations">Fit Run 2 stall strength to reproduce measured single mutant YFP rates for Run 3 initiation mutant simulations</a></li><li><a href="#Fit-Run-2-stall-strength-to-reproduce-measured-single-mutant-YFP-rates-for-Run-4-double-mutant-simulations">Fit Run 2 stall strength to reproduce measured single mutant YFP rates for Run 4 double mutant simulations</a></li><li><a href="#Fit-Run-2-stall-strength-to-reproduce-measured-single-mutant-YFP-rates-for-Run-5-CTC-distance-mutant-simulations">Fit Run 2 stall strength to reproduce measured single mutant YFP rates for Run 5 CTC distance mutant simulations</a></li><li><a href="#Fit-Run-2-stall-strength-to-reproduce-measured-single-mutant-YFP-rates-for-Run-11-CTA-distance-mutant-simulations">Fit Run 2 stall strength to reproduce measured single mutant YFP rates for Run 11 CTA distance mutant simulations</a></li><li><a href="#Fit-Run-13-stall-strength-to-reproduce-measured-single-mutant-YFP-rates-during-serine-starvation-for-Run-14-initiation-mutant-simulations">Fit Run 13 stall strength to reproduce measured single mutant YFP rates during serine starvation for Run 14 initiation mutant simulations</a></li><li><a href="#Fit-Run-13-stall-strength-to-reproduce-measured-single-mutant-YFP-rates-during-serine-starvation-for-Run-15-double-mutant-simulations">Fit Run 13 stall strength to reproduce measured single mutant YFP rates during serine starvation for Run 15 double mutant simulations</a></li><li><a href="#Fit-Run-2-stall-strength-to-reproduce-measured-single-mutant-YFP-rates-for-Run-16-multiple-CTA-mutant-simulations">Fit Run 2 stall strength to reproduce measured single mutant YFP rates for Run 16 multiple CTA mutant simulations</a></li></ol></div></div>

### Globals


In [2]:
import pandas as pd
import re
import os
import numpy as np
import simulation_utils
from scipy.interpolate import interp1d

### Fit Run 2 stall strength to reproduce measured single mutant YFP rates for Run 3 initiation mutant simulations

In [3]:
experimentdata = pd.read_table(
    '../processeddata/platereader/measured_yfprates_for_initiation_simulations.tsv',
    sep='\t',
    index_col=0)
'''
# Uncomment this region if run_simulations_whole_cell_parameter_sweep.ipynb
# was run to generate new simulation data

simulationdata = simulation_utils.get_simulation_data(runnumber=2)

simulationdata.drop(
    ['files'], axis=1).to_csv(
        '../rawdata/simulations/run2_data.tsv', sep='\t', index_label='index')
'''

simulationdata = pd.read_table(
    '../rawdata/simulations/run2_data.tsv', index_col=0)

pretermtypes = ['5primepreterm', 'selpreterm']
for pretermtype in pretermtypes:
    pretermrates = np.unique(simulationdata[pretermtype])
    for pretermrate in pretermrates:
        fitresults = dict()
        if pretermtype == 'selpreterm' and pretermrate == 0:
            continue
        for mutant in experimentdata.index:
            subset = simulationdata[(simulationdata[pretermtype] == pretermrate
                                     ) & (simulationdata['mutant'] == mutant.
                                          lower())]
            # if pretermrate is 0, make sure all other preterm rates are also 0
            if pretermrate == 0:
                for innerpretermtype in pretermtypes:
                    if innerpretermtype == pretermtype:
                        continue
                    subset = subset[(subset[innerpretermtype] == 0)]
            # to avoid parameter ranges that do not have any effect at pause
            # sites
            subset = subset[subset['ps_ratio'] < 0.9].sort_values(
                by=['stallstrength'])
            fit = interp1d(subset['ps_ratio'], subset['stallstrength'])
            temp = fit(experimentdata.ix[mutant]['measuredRateNormalized'])
            fitresults[mutant] = {'stallstrength': temp}

        if pretermrate == 0:
            title = 'trafficjam'
        else:
            title = pretermtype

        fitresults = pd.DataFrame.from_dict(fitresults, orient='index')
        fitresults.to_csv(
            '../processeddata/simulations/run2_fit_stallstrength_for_initiation_'
            + title + '.tsv',
            sep='\t',
            index_label='mutant')

### Fit Run 2 stall strength to reproduce measured single mutant YFP rates for Run 4 double mutant simulations

In [7]:
experimentdata = pd.read_table(
    '../processeddata/platereader/measured_yfprates_for_double_simulations.tsv',
    sep='\t',
    index_col=0)
'''
# Uncomment this region if run_simulations_whole_cell_parameter_sweep.ipynb
# was run to generate new simulation data

simulationdata = simulation_utils.get_simulation_data(runnumber=2)
'''

simulationdata = pd.read_table(
    '../rawdata/simulations/run2_data.tsv', index_col=0)

pretermtypes = ['5primepreterm', 'selpreterm']
for pretermtype in pretermtypes:
    pretermrates = np.unique(simulationdata[pretermtype])
    for pretermrate in pretermrates:
        fitresults = dict()
        if pretermtype == 'selpreterm' and pretermrate == 0:
            continue
        for mutant in experimentdata.index:
            subset = simulationdata[(simulationdata[pretermtype] == pretermrate
                                     ) & (simulationdata['mutant'] == mutant)]
            # if pretermrate is 0, make sure all other preterm rates are also 0
            if pretermrate == 0:
                for innerpretermtype in pretermtypes:
                    if innerpretermtype == pretermtype:
                        continue
                    subset = subset[(subset[innerpretermtype] == 0)]
            # to avoid parameter ranges that do not have any effect at pause
            # sites
            subset = subset[subset['ps_ratio'] < 0.9].sort_values(
                by=['stallstrength'])
            fit = interp1d(subset['ps_ratio'], subset['stallstrength'])
            temp = fit(experimentdata.ix[mutant]['measuredRateNormalized'])
            fitresults[mutant] = {'stallstrength': temp}

        if pretermrate == 0:
            title = 'trafficjam'
        else:
            title = pretermtype

        fitresults = pd.DataFrame.from_dict(fitresults, orient='index')
        fitresults.to_csv(
            '../processeddata/simulations/run2_fit_stallstrength_for_double_' +
            title + '.tsv',
            sep='\t',
            index_label='mutant')

### Fit Run 2 stall strength to reproduce measured single mutant YFP rates for Run 5 CTC distance mutant simulations

In [10]:
experimentdata = pd.read_table(
    '../processeddata/platereader/measured_yfprates_for_distance_simulations.tsv',
    sep='\t',
    index_col=0)
'''
# Uncomment this region if run_simulations_whole_cell_parameter_sweep.ipynb
# was run to generate new simulation data

simulationdata = simulation_utils.get_simulation_data(runnumber=2)
'''

simulationdata = pd.read_table(
    '../rawdata/simulations/run2_data.tsv', index_col=0)

pretermtypes = ['5primepreterm', 'selpreterm']
for pretermtype in pretermtypes:
    pretermrates = np.unique(simulationdata[pretermtype])
    for pretermrate in pretermrates:
        fitresults = dict()
        if pretermtype == 'selpreterm' and pretermrate == 0:
            continue
        for mutant in experimentdata.index:
            subset = simulationdata[(simulationdata[pretermtype] == pretermrate
                                     ) & (simulationdata['mutant'] == mutant.
                                          lower())]
            # if pretermrate is 0, make sure all other preterm rates are also 0
            if pretermrate == 0:
                for innerpretermtype in pretermtypes:
                    if innerpretermtype == pretermtype:
                        continue
                    subset = subset[(subset[innerpretermtype] == 0)]
            # to avoid parameter ranges that do not have any effect at pause
            # sites
            subset = subset[subset['ps_ratio'] < 0.9].sort_values(
                by=['stallstrength'])
            fit = interp1d(subset['ps_ratio'], subset['stallstrength'])
            temp = fit(experimentdata.ix[mutant]['measuredRateNormalized'])
            fitresults[mutant] = {'stallstrength': temp}

        if pretermrate == 0:
            title = 'trafficjam'
        else:
            title = pretermtype

        fitresults = pd.DataFrame.from_dict(fitresults, orient='index')
        fitresults.to_csv(
            '../processeddata/simulations/run2_fit_stallstrength_for_ctc_distance_'
            + title + '.tsv',
            sep='\t',
            index_label='mutant')

### Fit Run 2 stall strength to reproduce measured single mutant YFP rates for Run 11 CTA distance mutant simulations

In [11]:
experimentdata = pd.read_table(
    '../processeddata/platereader/measured_yfprates_for_cta_distance_simulations.tsv',
    sep='\t',
    index_col=0)
'''
# Uncomment this region if run_simulations_whole_cell_parameter_sweep.ipynb
# was run to generate new simulation data

simulationdata = simulation_utils.get_simulation_data(runnumber=2)
'''

simulationdata = pd.read_table(
    '../rawdata/simulations/run2_data.tsv', index_col=0)

pretermtypes = ['5primepreterm', 'selpreterm']
for pretermtype in pretermtypes:
    pretermrates = np.unique(simulationdata[pretermtype])
    for pretermrate in pretermrates:
        fitresults = dict()
        if pretermtype == 'selpreterm' and pretermrate == 0:
            continue
        for mutant in experimentdata.index:
            subset = simulationdata[(simulationdata[pretermtype] == pretermrate
                                     ) & (simulationdata['mutant'] == mutant.
                                          lower())]
            # if pretermrate is 0, make sure all other preterm rates are also 0
            if pretermrate == 0:
                for innerpretermtype in pretermtypes:
                    if innerpretermtype == pretermtype:
                        continue
                    subset = subset[(subset[innerpretermtype] == 0)]
            # to avoid parameter ranges that do not have any effect at pause
            # sites
            subset = subset[subset['ps_ratio'] < 0.9].sort_values(
                by=['stallstrength'])
            fit = interp1d(subset['ps_ratio'], subset['stallstrength'])
            temp = fit(experimentdata.ix[mutant]['measuredRateNormalized'])
            fitresults[mutant] = {'stallstrength': temp}

        if pretermrate == 0:
            title = 'trafficjam'
        else:
            title = pretermtype

        fitresults = pd.DataFrame.from_dict(fitresults, orient='index')
        fitresults.to_csv(
            '../processeddata/simulations/run2_fit_stallstrength_for_cta_distance_'
            + title + '.tsv',
            sep='\t',
            index_label='mutant')

### Fit Run 13 stall strength to reproduce measured single mutant YFP rates during serine starvation for Run 14 initiation mutant simulations

In [12]:
experimentdata = pd.read_table(
    '../processeddata/platereader/measured_yfprates_for_serine_initiation_simulations.tsv',
    sep='\t',
    index_col=0)
'''
# Uncomment this region if run_simulations_whole_cell_parameter_sweep.ipynb
# was run to generate new simulation data

simulationdata = simulation_utils.get_simulation_data(runnumber=13)

simulationdata.drop(
    ['files'], axis=1).to_csv(
        '../rawdata/simulations/run13_data.tsv', sep='\t', index_label='index')
'''

simulationdata = pd.read_table(
    '../rawdata/simulations/run13_data.tsv', index_col=0)

pretermtypes = ['5primepreterm', 'selpreterm']
for pretermtype in pretermtypes:
    pretermrates = np.unique(simulationdata[pretermtype])
    for pretermrate in pretermrates:
        fitresults = dict()
        if pretermtype == 'selpreterm' and pretermrate == 0:
            continue
        for mutant in experimentdata.index:
            subset = simulationdata[(simulationdata[pretermtype] == pretermrate
                                     ) & (simulationdata['mutant'] == mutant.
                                          lower())]
            # if pretermrate is 0, make sure all other preterm rates are also 0
            if pretermrate == 0:
                for innerpretermtype in pretermtypes:
                    if innerpretermtype == pretermtype:
                        continue
                    subset = subset[(subset[innerpretermtype] == 0)]
            # to avoid parameter ranges that do not have any effect at pause
            # sites
            subset = subset[subset['ps_ratio'] < 0.9].sort_values(
                by=['stallstrength'])
            fit = interp1d(subset['ps_ratio'], subset['stallstrength'])
            temp = fit(experimentdata.ix[mutant]['measuredRateNormalized'])
            fitresults[mutant] = {'stallstrength': temp}

        if pretermrate == 0:
            title = 'trafficjam'
        else:
            title = pretermtype

        fitresults = pd.DataFrame.from_dict(fitresults, orient='index')
        fitresults.to_csv(
            '../processeddata/simulations/run13_serine_fit_stallstrength_for_initiation_'
            + title + '.tsv',
            sep='\t',
            index_label='mutant')

### Fit Run 13 stall strength to reproduce measured single mutant YFP rates during serine starvation for Run 15 double mutant simulations

In [19]:
experimentdata = pd.read_table(
    '../processeddata/platereader/measured_yfprates_for_serine_double_simulations.tsv',
    sep='\t',
    index_col=0)
'''
# Uncomment this region if run_simulations_whole_cell_parameter_sweep.ipynb
# was run to generate new simulation data

simulationdata = simulation_utils.get_simulation_data(runnumber=13)
'''

simulationdata = pd.read_table(
    '../rawdata/simulations/run13_data.tsv', index_col=0)

pretermtypes = ['5primepreterm', 'selpreterm']
for pretermtype in pretermtypes:
    pretermrates = np.unique(simulationdata[pretermtype])
    for pretermrate in pretermrates:
        fitresults = dict()
        if pretermtype == 'selpreterm' and pretermrate == 0:
            continue
        for mutant in experimentdata.index:
            if mutant == 'tcg8':  # I did not use TCG8 for double mutants
                continue
            subset = simulationdata[(simulationdata[pretermtype] == pretermrate
                                     ) & (simulationdata['mutant'] == mutant.
                                          lower())]
            # if pretermrate is 0, make sure all other preterm rates are also 0
            if pretermrate == 0:
                for innerpretermtype in pretermtypes:
                    if innerpretermtype == pretermtype:
                        continue
                    subset = subset[(subset[innerpretermtype] == 0)]
            # to avoid parameter ranges that do not have any effect at pause
            # sites
            subset = subset[subset['ps_ratio'] < 0.9].sort_values(
                by=['stallstrength'])
            fit = interp1d(subset['ps_ratio'], subset['stallstrength'])
            temp = fit(experimentdata.ix[mutant]['measuredRateNormalized'])
            fitresults[mutant] = {'stallstrength': temp}

        if pretermrate == 0:
            title = 'trafficjam'
        else:
            title = pretermtype

        fitresults = pd.DataFrame.from_dict(fitresults, orient='index')
        fitresults.to_csv(
            '../processeddata/simulations/run13_serine_fit_stallstrength_for_double_'
            + title + '.tsv',
            sep='\t',
            index_label='mutant')

### Fit Run 2 stall strength to reproduce measured single mutant YFP rates for Run 16 multiple CTA mutant simulations

In [16]:
experimentdata = pd.read_table(
    '../processeddata/platereader/measured_yfprates_for_leucine_multiple_simulations.tsv',
    sep='\t',
    index_col=0)
'''
# Uncomment this region if run_simulations_whole_cell_parameter_sweep.ipynb
# was run to generate new simulation data

simulationdata = simulation_utils.get_simulation_data(runnumber=2)
'''

simulationdata = pd.read_table(
    '../rawdata/simulations/run2_data.tsv', index_col=0)

pretermtypes = ['5primepreterm', 'selpreterm']
for pretermtype in pretermtypes:
    pretermrates = np.unique(simulationdata[pretermtype])
    for pretermrate in pretermrates:
        fitresults = dict()
        if pretermtype == 'selpreterm' and pretermrate == 0:
            continue
        for mutant in experimentdata.index:
            subset = simulationdata[(simulationdata[pretermtype] == pretermrate
                                     ) & (simulationdata['mutant'] == mutant)]
            # if pretermrate is 0, make sure all other preterm rates are also 0
            if pretermrate == 0:
                for innerpretermtype in pretermtypes:
                    if innerpretermtype == pretermtype:
                        continue
                    subset = subset[(subset[innerpretermtype] == 0)]
            # to avoid parameter ranges that do not have any effect at pause
            # sites
            subset = subset[subset['ps_ratio'] < 0.9].sort_values(
                by=['stallstrength'])
            fit = interp1d(subset['ps_ratio'], subset['stallstrength'])
            temp = fit(experimentdata.ix[mutant]['measuredRateNormalized'])
            fitresults[mutant] = {'stallstrength': temp}

        if pretermrate == 0:
            title = 'trafficjam'
        else:
            title = pretermtype

        fitresults = pd.DataFrame.from_dict(fitresults, orient='index')
        fitresults.to_csv(
            '../processeddata/simulations/run2_fit_stallstrength_for_leucine_multiple_'
            + title + '.tsv',
            sep='\t',
            index_label='mutant')